# Obtaining Public Water Service rates

The PWS rates information is given in the challenge dataset #10. After looking at it and realizing that only a small fraction of the providers had usable information, we tried to find a more complete source for this data.

Following Greg's suggestions, we found the data in a report titled "Water Rate Survey 2017", authored by the American Water Works Association (California-Nevada section), Raftelis, and California Data Collaborative. The following evening, we were notified that the same table was also available as a CSV file. Since I had already analyzed the PDF, I decided it could be useful knowledge for other circumstances where the machine-readable data is not available.

In [1]:
from tabula import read_pdf
import pandas as pd

In [1]:
df = read_pdf('CA-NV_RateSurvey-2017_final.pdf', pages='19-27')
df.head()

,Unnamed: 0,Effective,Unnamed: 2,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
0,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge
1,Alameda County Water District,3/1/2018,Bi-Monthly,ccf,9,Uniform,26.16,36.49,62.66
2,Amador Water Agency,10/1/2017,Monthly,ccf,8,Uniform,25.08,20.64,45.72
3,American Canyon City Of,6/1/2017,Monthly,ccf,9,Tiered,6.4,49.23,55.63
4,Anaheim City of,2/1/2016,Monthly,ccf,10,Uniform,12.97,5.19,18.16


We can see that there are some spurious header rows among the real data.
Usually, we can pass the line numbers to skip to the `read_csv()` function (in Tabula-py, exposed through the `pandas_option` kwarg parameter). This doesn't work here because the header is repeated on each page of the PDF; Tabula-py seems to be applying `pandas_options` only to the full (multipage) intermediate CSV table.

In [66]:
def read_pdf_skip_header(path, pages):
    return read_pdf(path, pages=pages, pandas_options={'skiprows': [1]})

def is_spurious_header(df):
    # these values are just examples, any other column would work
    return df['Effective'].isin(['Effective', 'Date'])

def remove_spurious_header_rows(df):
    rows = df.loc[is_spurious_header].index
    return df.drop(rows)

In [18]:
(read_pdf_skip_header('CA-NV_RateSurvey-2017_final.pdf', pages='19-27')
 .loc[is_spurious_header]
)

,Unnamed: 0,Effective,Unnamed: 2,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
37,NaN,Effective,NaN,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
38,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge
76,NaN,Effective,NaN,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
77,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge
115,NaN,Effective,NaN,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
116,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge
154,NaN,Effective,NaN,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
155,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge
192,NaN,Effective,NaN,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
193,Water Service Provider,Date,Bill Frequency,Unit,Custom Usage,Structure,Charge,Charge,Charge




We could select the spurious header rows and then reject them, based on ad-hoc values:

In [20]:
def remove_spurious_header_rows(df):
    rows = df.loc[is_spurious_header].index
    return df.drop(rows)

In [20]:
(read_pdf_skip_header('CA-NV_RateSurvey-2017_final.pdf', pages='19-27')
 .pipe(remove_spurious_header_rows)
 .loc[is_spurious_header]
)

,Unnamed: 0,Effective,Unnamed: 2,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total


A more solid approach would be to implement the multi-page reading functionality ourselves, so that `pandas_options` is applied on each page as intended, and then concatenating the single-page tables using `pd.concat`.
For the general case, this approach also offers the possibility of implementing per-page adjustments, if necessary.
It is possible that this is slower for very large tables, so flexibility/performance tradeoffs should be considered.

In [60]:
def read_pdf_separate_pages(path, pages, **pandas_options):
     return pd.concat((read_pdf(path, pages=[page],
                                pandas_options=pandas_options)
                       for page in pages), ignore_index=True)

In [61]:
df = read_pdf_separate_pages('CA-NV_RateSurvey-2017_final.pdf', pages=range(17, 28), skiprows=[1])
df.loc[is_spurious_header]

,Unnamed: 0,Effective,Unnamed: 2,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total


In [59]:
df

,Unnamed: 0,Effective,Unnamed: 2,Billing,Average Monthly,Rate,Monthly Fixed,Monthly Variable,Total
0,Alameda County Water District,3/1/2018,Bi-Monthly,ccf,9,Uniform,26.16,36.49,62.66
1,Amador Water Agency,10/1/2017,Monthly,ccf,8,Uniform,25.08,20.64,45.72
2,American Canyon City Of,6/1/2017,Monthly,ccf,9,Tiered,6.40,49.23,55.63
3,Anaheim City of,2/1/2016,Monthly,ccf,10,Uniform,12.97,5.19,18.16
4,Antioch City Of,7/1/2017,Monthly,ccf,10,Tiered,21.20,32.52,53.72
5,Apple Valley Ranchos Water Company,1/1/2017,monthly,ccf,15,Tiered,34.73,63.14,97.87
6,Arcadia City Of,4/1/2017,Bimonthly,ccf,19,Tiered,10.17,32.52,42.69
7,Arcata City Of,10/1/2017,Monthly,ccf,4,Tiered,12.16,14.09,26.25
8,Arrowbear Park County Water District,12/19/2016,Monthly,ccf,15,Tiered,27.50,58.50,86.00
9,Arroyo Grande City Of,8/9/2017,Monthly,ccf,9,Tiered,34.34,36.60,70.94


In [39]:
COLUMNS = [
    'provider_name',
    'effective_date',
    'billing_freq',
    'billing_unit',
    'avg_monthly_custom_usage',
    'rate_structure',
    'monthly_fixed_charge',
    'monthly_variable_charge',
    'total_charge'
]

CURRENCY_TYPE = float  # never do this in production!!!

DTYPES = {
    'provider_name': 'category',
    'billing_freq': 'category',
    'billing_unit': 'category',
    'avg_monthly_custom_usage': int,  # TODO we should check this
    'rate_structure': 'category',
    'monthly_fixed_charge': CURRENCY_TYPE,
    'monthly_variable_charge': CURRENCY_TYPE,
    'total_charge': CURRENCY_TYPE,   
}

The pandas Categorical dtype is roughly equivalent to an enum, and should be used whenever values can only take a finite number of options. The `uniformize_entries` is needed to normalize the various options for columns such as `billing_freq`, `billing_unit` and `rate_structure`, where the input values were formatted inconsistently.

In [50]:
def uniformize_entries(col):
    return (col
            .astype(str)
            .str.replace('-', '')
            .str.title()
           )

def clean(df):
    # never modify in-place!
    df = df.copy()
    df.columns = COLUMNS
    return (df
            .assign(rate_structure=lambda d: uniformize_entries(d.rate_structure),
                    billing_freq=lambda d: uniformize_entries(d.billing_freq),
                    provider_name=lambda d: uniformize_entries(d.provider_name),
                    effective_date=lambda d: pd.to_datetime(d.effective_date))
            .astype(DTYPES)
           )

In [62]:
df = clean(df)
df

,provider_name,effective_date,billing_freq,billing_unit,avg_monthly_custom_usage,rate_structure,monthly_fixed_charge,monthly_variable_charge,total_charge
0,Alameda County Water District,2018-03-01,Bimonthly,ccf,9,Uniform,26.16,36.49,62.66
1,Amador Water Agency,2017-10-01,Monthly,ccf,8,Uniform,25.08,20.64,45.72
2,American Canyon City Of,2017-06-01,Monthly,ccf,9,Tiered,6.40,49.23,55.63
3,Anaheim City Of,2016-02-01,Monthly,ccf,10,Uniform,12.97,5.19,18.16
4,Antioch City Of,2017-07-01,Monthly,ccf,10,Tiered,21.20,32.52,53.72
5,Apple Valley Ranchos Water Company,2017-01-01,Monthly,ccf,15,Tiered,34.73,63.14,97.87
6,Arcadia City Of,2017-04-01,Bimonthly,ccf,19,Tiered,10.17,32.52,42.69
7,Arcata City Of,2017-10-01,Monthly,ccf,4,Tiered,12.16,14.09,26.25
8,Arrowbear Park County Water District,2016-12-19,Monthly,ccf,15,Tiered,27.50,58.50,86.00
9,Arroyo Grande City Of,2017-08-09,Monthly,ccf,9,Tiered,34.34,36.60,70.94


# TODO

- Join with the Water Provider info table (`WPSID`, etc)


# Notes
- This table probably has cities from Nevada as well


In [65]:
from cawc import PATH_DATA
PATH_DATA

PosixPath('/data/datasets/cawc')